In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import os
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import tree
warnings.filterwarnings('ignore')

In [ ]:
# Load datasets
players_df = pd.read_csv('./datasets/players.csv')
teams_df = pd.read_csv('./datasets/teams.csv')

# Preview the datasets
print("Players Data Overview:")
print(players_df.info())
print(players_df.head(), "\n")

print("Teams Data Overview:")
print(teams_df.info())
print(teams_df.head(), "\n")

In [ ]:
# Ensure uniqueness of teams in both datasets
print("\nUnique teams in Players Dataset:", players_df['Team'].unique())
print("Unique teams in Teams Dataset:", teams_df['Team'].unique())

# Check for duplicates and drop them
print("\nDuplicates in Players Dataset:", players_df.duplicated().sum())
print("Duplicates in Teams Dataset:", teams_df.duplicated().sum())
players_df = players_df.drop_duplicates()
teams_df = teams_df.drop_duplicates()

In [ ]:
# # Missing values
# print("\nMissing Values in Players Dataset:")
# print(players_df.isnull().sum())
# print("\nMissing Values in Teams Dataset:")
# print(teams_df.isnull().sum())
#
# # Fill missing values (example: fill numeric with median, categorical with mode)
# players_df.fillna(players_df.median(numeric_only=True), inplace=True)
# teams_df.fillna(teams_df.median(numeric_only=True), inplace=True)
# players_df.fillna(players_df.mode().iloc[0], inplace=True)
# teams_df.fillna(teams_df.mode().iloc[0], inplace=True)

In [ ]:
# Merge datasets on the common column 'Team' 
combined_df = pd.merge(players_df, teams_df, on='Team', how='inner')
print("\nCombined Data Overview:")
print(combined_df.info())
print(combined_df.head())

# Descriptive Statistics
print("\nDescriptive Statistics for Players Data:")
print(players_df.describe())

print("\nDescriptive Statistics for Teams Data:")
print(teams_df.describe())

In [ ]:
# Data Visualization
sns.set_theme(style="whitegrid")

# Histogram of player points
plt.figure(figsize=(10, 6))
sns.histplot(players_df['PTS'], bins=20, kde=True, color='blue')
plt.title('Distribution of Player Points')
plt.xlabel('Points')
plt.ylabel('Freq')
plt.show()

In [ ]:
teams_df.head()
# Team vs Player points comparison
team_points = teams_df[['Team', 'PTS']].set_index('Team')
player_avg_points = players_df.groupby('Team')['PTS'].mean()

plt.figure(figsize=(14, 7))
team_points['Average Player Points'] = player_avg_points
team_points.plot(kind='bar', figsize=(14, 7), title='Team vs Player Points Comparison')
plt.ylabel('pts')
plt.show()

In [ ]:
# # Correlation heatmap
# plt.figure(figsize=(10, 8))
# corr = combined_df.corr()
# sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True)
# plt.title('Correlation Heatmap')
# plt.show()

In [ ]:
# Outlier detection using IQR (detects points that are far or different from other points in the dataset)
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

outliers = detect_outliers(players_df, 'PTS')
print("\nOutliers in Player Points:")
print(outliers)

In [ ]:
# Save cleaned and processed data
players_df.to_csv('./datasets/cleaned_players.csv', index=False)
teams_df.to_csv('./datasets/cleaned_teams.csv', index=False)

print("\nEDA completed. Cleaned datasets saved!")

In [ ]:
# Identify categorical columns
categorical_cols_players = players_df.select_dtypes(include=['object']).columns
categorical_cols_teams = teams_df.select_dtypes(include=['object']).columns

print("\nCategorical Columns in Players Dataset:", categorical_cols_players.tolist())
print("Categorical Columns in Teams Dataset:", categorical_cols_teams.tolist())

# # Handle missing values in categorical columns (fill with mode)
# for col in categorical_cols_players:
#     players_df[col].fillna(players_df[col].mode()[0], inplace=True)
# 
# for col in categorical_cols_teams:
#     teams_df[col].fillna(teams_df[col].mode()[0], inplace=True)

In [ ]:
# Feature engineering: Create new features from categorical data (here convert team names to team codes)
team_mapping = {name: idx for idx, name in enumerate(players_df['Team'].unique())}
players_df['team_code'] = players_df['Team'].map(team_mapping)
teams_df['team_code'] = teams_df['Team'].map(team_mapping)

In [ ]:
# Encoding: Convert categorical variables into numerical formats using one-hot encoding technique
players_encoded = pd.get_dummies(players_df, columns=['Name', 'Team'], prefix='pos', drop_first=True)
teams_encoded = pd.get_dummies(teams_df, columns=['Team'], prefix='conf', drop_first=True)

In [ ]:
# Scaling: Apply standardization for consistency (Scale player stats (excluding categorical/team_code))
numerical_cols_players = players_df.select_dtypes(include=['float64', 'int64']).columns
scaler_players = StandardScaler()
players_df[numerical_cols_players] = scaler_players.fit_transform(players_df[numerical_cols_players])
numerical_cols_teams = teams_df.select_dtypes(include=['float64', 'int64']).columns
scaler_teams = StandardScaler()
teams_df[numerical_cols_teams] = scaler_teams.fit_transform(teams_df[numerical_cols_teams])

In [ ]:
# Final combined DataFrame for preprocessing
final_combined_df = pd.merge(players_encoded, teams_encoded, on='team_code', how='inner')
print("\nPreprocessed Combined DataFrame Overview:")
print(final_combined_df.info())
print(final_combined_df.head())
final_combined_df.to_csv('./datasets/data.csv', index=False)

In [ ]:
preprocessing_steps = """
### Data Preprocessing Documentation

#### 1. Raw Data Cleaning:
- Dropped duplicate rows in both datasets.
- Standardized column names to lowercase and stripped extra spaces.
- Filled missing values:
  - **Numeric columns**: Imputed using the median.
  - **Categorical columns**: Imputed using the mode.

#### 2. Handling Categorical Variables:
- Identified categorical columns: 'team', 'position', and 'conference'.
- Applied the following techniques:
  - **Imputation**: Filled missing values with the most frequent value.
  - **Feature Engineering**: Created a 'team_code' column by mapping team names to unique numeric codes.
  - **Encoding**: One-hot encoded the 'position' (players) and 'conference' (teams) columns.

#### 3. Scaling:
- Standardized all numerical features using `StandardScaler` to ensure uniformity.

#### 4. Data Merging:
- Merged the players and teams datasets on the newly created 'team_code' column for seamless integration.

#### 5. Final Dataset:
- Saved the cleaned and processed datasets as 'cleaned_players.csv', 'cleaned_teams.csv', and 'final_combined.csv' for future use.
"""

# Save the preprocessing steps to a text file
with open("preprocessing_documentation.txt", "w") as file:
    file.write(preprocessing_steps)

print("\nPreprocessing steps documented and saved to 'preprocessing_documentation.txt'.")

In [ ]:
# Select features and target for regression (here predicting 'points' according to other features)
X = players_df[['AST', 'REB', 'STL', 'BLK', 'TOV', 'Min', 'Age', 'GP', 'W', 'L', 'FG%']]
y = players_df['PTS']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize Linear Regression model
lin_reg = LinearRegression()

# Fit model to training data
lin_reg.fit(X_train, y_train)

# Predict on the test data
y_pred = lin_reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Plot predicted vs actual points
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.title('Linear Regression: Predicted vs Actual Points')
plt.xlabel('Actual Points')
plt.ylabel('Predicted Points')
plt.show()

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

In [ ]:
# Create a binary classification target based on points (high vs low performance)
median_points = players_df['PTS'].median()
players_df['performance'] = (players_df['PTS'] >= median_points).astype(int)  # 1 for High, 0 for Low

# Select features (excluding 'points' and 'performance')
X_classify = players_df[['AST', 'REB', 'STL', 'BLK', 'TOV', 'Min', 'Age', 'GP', 'W', 'L', 'FG%']]
y_classify = players_df['performance']

# Split data into training and test sets
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_classify, y_classify, test_size=0.2, random_state=42)
# Initialize Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Fit model to training data
clf.fit(X_train_class, y_train_class)

# Predict on the test data
y_pred_class = clf.predict(X_test_class)

# Evaluate the classifier
accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"Accuracy: {accuracy}")
print("\nClassification Report:\n", classification_report(y_test_class, y_pred_class))

# Visualize the decision tree
plt.figure(figsize=(12,8))
tree.plot_tree(clf, feature_names=X_classify.columns, class_names=["Low", "High"], filled=True, rounded=True)
plt.title('Decision Tree Classifier for Player Performance')
plt.show()

In [ ]:
# Initialize ANN (MLP Classifier)
ann_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Train the ANN model
ann_clf.fit(X_train_class, y_train_class)

# Predict and evaluate
y_pred_ann = ann_clf.predict(X_test_class)
accuracy_ann = accuracy_score(y_test_class, y_pred_ann)

print(f"ANN Accuracy: {accuracy_ann}")
print("\nANN Classification Report:\n", classification_report(y_test_class, y_pred_ann))

In [ ]:
# Get the confusion matrix for the classification model
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Low", "High"])

# Plot confusion matrix
cm_display.plot(cmap="Blues")
plt.title("Confusion Matrix for Decision Tree / ANN Classifier")
plt.show()

# Display classification accuracy
accuracy_class = accuracy_score(y_test_class, y_pred_class)
print(f"Accuracy: {accuracy_class}")

In [ ]:
# Predicted probabilities for each class
y_pred_prob = ann_clf.predict_proba(X_test_class)

# Plot predicted probabilities for the 'High' class
plt.figure(figsize=(8,6))
plt.hist(y_pred_prob[:, 1], bins=10, color='purple', alpha=0.7, label='Predicted Probability for High Performance')
plt.title('Predicted Probabilities for High Performance')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Evaluate the regression model using MSE, RMSE, and R²
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")
# Evaluate the classification model
accuracy = accuracy_score(y_test_class, y_pred_class)
precision = precision_score(y_test_class, y_pred_class)
recall = recall_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test_class, y_pred_class))

In [ ]:
# Save the models using pkl
os.makedirs('./models', exist_ok=True)

# Save the models using pickle
with open('./models/ann_clf.pkl', 'wb') as file:
    pickle.dump(ann_clf, file)

with open('./models/ling_reg.pkl', 'wb') as file:
    pickle.dump(lin_reg, file)

with open('./models/clf.pkl', 'wb') as file:
    pickle.dump(clf, file)  